In [1]:
# smo
import pandas as pd

bulkload_sizes = [0, 1000000, 2000000, 5000000, 10000000, 20000000, 50000000, 100000000]
sizes2ratios = {
    0: "0.0",
    1000000: "0.005",
    2000000: "0.01",
    5000000: "0.025",
    10000000: "0.05",
    20000000: "0.1",
    50000000: "0.25",
    100000000: "0.5",
}
datasets = ["linear", "covid", "fb-1", "osm"]

df = pd.read_csv("../data/AIO_trimed.csv")
df = df[(df["read_ratio"] != 0)]
df['key_path'] = df['key_path'].apply(lambda x: x.replace('fb', 'fb-1'))
df_grouped = df.groupby(['index_type', 'key_path', 'test_suite', 'init_table_size'])['throughput'].mean().reset_index()

result = {}

for index in ["alex", "lipp"]:
    smo_counter = 0
    counter = 0
    for case1 in [21, 41]:
        for dataset in datasets:
            for bulkload_size in bulkload_sizes:
                    if case1 == 41 and bulkload_size == 0:
                        continue
                    
                    counter += 1

                    if case1 == 21:
                        case2 = 22
                    else:
                        case2 = 42
                    
                    # get case throughput
                    d1 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_size) & (df_grouped["test_suite"] == case1) & (df_grouped["index_type"] == index)]["throughput"].values
                    d2 = df_grouped[(df_grouped["key_path"] == f"datasets/{dataset}") & (df_grouped["init_table_size"] == bulkload_size) & (df_grouped["test_suite"] == case2) & (df_grouped["index_type"] == index)]["throughput"].values
                    f1 = f"../log/root_smo_profiling/{dataset}/{case1}/{sizes2ratios[bulkload_size]}/{index}_insert_smo_stats.log"
                    f2 = f"../log/root_smo_profiling/{dataset}/{case2}/{sizes2ratios[bulkload_size]}/{index}_insert_smo_stats.log"
                    p1 = pd.read_csv(f1)
                    p2 = pd.read_csv(f2)
                    smo_cnt1 = p1["count"].sum()
                    smo_cnt2 = p2["count"].sum()
                    if (smo_cnt1 > smo_cnt2 and d1[0] < d2[0]) or (smo_cnt1 < smo_cnt2 and d1[0] > d2[0]):
                        smo_counter += 1
                        # print(f"index: {index}, {case1} vs. {case2}, [{smo_cnt1} vs. {smo_cnt2}]. dataset:{dataset}, bulkload_size: {bulkload_size}, {d1}/{d2}={d1/d2}")
    result[index] = (smo_counter, counter)

# print(f"smo_counter: {smo_counter}, counter: {counter}, ratio: {smo_counter/counter}")
for index in result:
    print(f"index: {index}, smo_counter: {result[index][0]}, counter: {result[index][1]}, ratio: {result[index][0]/result[index][1]}")

index: alex, smo_counter: 50, counter: 60, ratio: 0.8333333333333334
index: lipp, smo_counter: 49, counter: 60, ratio: 0.8166666666666667
